# Comparison of spectrum between eNATL60 outputs and Alitka Satellite 

Requisites :
 - git clone https://github.com/auraoupa/gonzag_cloud
 - git clone https://github.com/brodeau/climporn
 
Assumes that time is encoded in datetime64 format in both datasets.

Looks for model months in the longest year of Altika serie

In [1]:
import sys,os
from os import getenv
import warnings
warnings.filterwarnings("ignore")

In [2]:
GONZAG_DIR = '/home/jovyan/gonzag_cloud/gonzag' ; # get it there: https://github.com/brodeau/climporn
sys.path.append(GONZAG_DIR)
import gonzag as gz


## Params

In [3]:
model = 'GIGATL'
region = '1'
season = 'fma'
name_mod = model+'-Region'+region+'-'+season
name_ssh_mod = 'zeta'
name_lsm_mod = 'salt' 
name_lat_mod = 'lat'
name_lon_mod = 'lon'
name_time_mod = 'ocean_time'
l_griddist = False

## Data

In [4]:
import xarray as xr
import intake
import gcsfs

gcs = gcsfs.GCSFileSystem(requester_pays=True)

cat = intake.open_catalog('../SWOT-AdAC/catalog.yaml')
dsmod = cat[model](region=region,depth='surf', season=season).to_dask()

In [5]:
dsmod

<xarray.Dataset>
Dimensions:     (eta_rho: 1489, eta_v: 1488, time: 2280, xi_rho: 1491, xi_u: 1490)
Dimensions without coordinates: eta_rho, eta_v, time, xi_rho, xi_u
Data variables: (12/13)
    angle       (eta_rho, xi_rho) float32 dask.array<chunksize=(1489, 1491), meta=np.ndarray>
    f           (eta_rho, xi_rho) float32 dask.array<chunksize=(1489, 1491), meta=np.ndarray>
    h           (eta_rho, xi_rho) float32 dask.array<chunksize=(1489, 1491), meta=np.ndarray>
    lat         (eta_rho, xi_rho) float32 dask.array<chunksize=(1489, 1491), meta=np.ndarray>
    lon         (eta_rho, xi_rho) float32 dask.array<chunksize=(1489, 1491), meta=np.ndarray>
    ocean_time  (time) float32 dask.array<chunksize=(24,), meta=np.ndarray>
    ...          ...
    pn          (eta_rho, xi_rho) float32 dask.array<chunksize=(1489, 1491), meta=np.ndarray>
    salt        (time, eta_rho, xi_rho) float32 dask.array<chunksize=(24, 1489, 1491), meta=np.ndarray>
    temp        (time, eta_rho, xi_rho) float32 dask.array<chunksize=(24, 1489, 1491), meta=np.ndarray>
    u           (time, eta_rho, xi_u) float32 dask.array<chunksize=(24, 1489, 1490), meta=np.ndarray>
    v           (time, eta_v, xi_rho) float32 dask.array<chunksize=(24, 1488, 1491), meta=np.ndarray>
    zeta        (time, eta_rho, xi_rho) float32 dask.array<chunksize=(24, 1489, 1491), meta=np.ndarray>

In [7]:
from intake import open_catalog
cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/ocean/altimetry.yaml")
dssat  = cat["al"].to_dask()

In [8]:
dssat

<xarray.Dataset>
Dimensions:         (time: 34340107)
Coordinates:
    latitude        (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    longitude       (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
  * time            (time) datetime64[ns] 2013-03-14T05:44:50.445185 ... 2015...
Data variables:
    cycle           (time) int16 dask.array<chunksize=(34340107,), meta=np.ndarray>
    dac             (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    lwe             (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    mdt             (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    ocean_tide      (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    sla_filtered    (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    sla_unfiltered  (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    track           (time) int16 dask.array<chunksize=(34340107,), meta=np.ndarray>
Attributes: (12/24)
    Conventions:               CF-1.6
    Metadata_Conventions:      Unidata Dataset Discovery v1.0
    cdm_data_type:             Swath
    comment:                   Sea surface height measured by altimeters refe...
    contact:                   servicedesk.cmems@mercator-ocean.eu
    creator_email:             servicedesk.cmems@mercator-ocean.eu
    ...                        ...
    software_version:          6.2_DUACS_DT2018_baseline
    source:                    Altika measurements
    ssalto_duacs_comment:      The reference mission used for the altimeter i...
    standard_name_vocabulary:  NetCDF Climate and Forecast (CF) Metadata Conv...
    summary:                   SSALTO/DUACS Delayed-Time Level-3 sea surface ...
    title:                     DT Altika Global Ocean Along track SSALTO/DUAC...

In [9]:
name_sat= 'Altika'
name_ssh_sat='sla_unfiltered'
name_time_sat='time'

## Time overlap between model and satellite ?
Assume that model is less than 1 year, satellite has at least one whole year, we take the overlapping period without considering which year is this

In [10]:
    (date1,date2), (Nts,Ntm) = gz.GetTimeOverlap( dssat , dsmod, name_time_sat, name_time_mod )
    print(' *** Time overlap between model and satellite in UNIX epoch time: it1, it2',date1,'--',date2)


 *** Time overlap between model and satellite in UNIX epoch time: it1, it2 2014-01-29 -- 2014-05-03


## Create object `ModelGrid` containing all the model (aka _source_) 2D+T domain grid info

In [11]:
  
    ModelGrid = gz.ModGrid( dsmod, date1 , date2, name_lon_mod, name_lat_mod , dsmod, name_lsm_mod, distorded_grid=False )


TypeError: GetTimeVector() missing 1 required positional argument: 'varname'

## Create object `SatelliteTrack` containing all the satellite track (aka target) info

In [ ]:
SatelliteTrack = gz.SatTrack( dssat, date1, date2, name_ssh_sat, Np=Nts, \
                              domain_bounds=ModelGrid.domain_bounds, l_0_360=ModelGrid.l360 )

## Process the tracks in parallel (results are saved in separate netcdf files)

In [ ]:
if not os.path.exists('results_'+name_sat+'-'+name_mod):
        os.makedirs('results_'+name_sat+'-'+name_mod)

In [ ]:
def process_one_track(track):
    tt = "{:02d}".format(track)
    Solution0 = gz.Model2SatTrack( ModelGrid, name_ssh_mod, SatelliteTrack, name_ssh_sat, track )
    c1     = 'Model SSH interpolated in space (' ; c2=') and time on satellite track'
    vvar   = [ 'latitude', 'longitude', name_ssh_mod+'_np'   , name_ssh_mod+'_bl' , name_ssh_sat          , 'distance'                            ]
    vunits = [ 'deg.N'   , 'deg.E'    ,          'm'         ,     'm'            ,    'm'                ,    'km'                               ]
    vlongN = [ 'Latitude', 'Longitude', c1+'nearest-point'+c2,  c1+'bilinear'+c2  , 'Input satellite data', 'Cumulated distance from first point' ]
    from gonzag.config import rmissval
    iw = gz.io.SaveTimeSeries( Solution0.time, \
                             nmp.array( [Solution0.lat, Solution0.lon, Solution0.ssh_mod_np,
                                         Solution0.ssh_mod, Solution0.ssh_sat, Solution0.distance] ), \
                             vvar, 'results_'+name_sat+'-'+name_mod+'/result_'+str(tt)+'.nc', time_units='',\
                             vunits=vunits, vlnm=vlongN, missing_val=rmissval )
    return Solution0


In [ ]:
import dask
import sys
import dask.bag as db
import numpy as nmp

In [ ]:
from dask.distributed import Client
client = Client()
client

In [ ]:
%%time
b = db.from_sequence(nmp.arange(len(SatelliteTrack.index_tracks)), npartitions=64)
results = b.map(lambda x: process_one_track(x))
final = results.compute()

## Plot the results

In [ ]:
import xarray as xr
import sys
import glob
import numpy as nmp
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker

%matplotlib inline


In [ ]:
CLIMPORN_DIR = '/home/jovyan/climporn/python' ; # get it there: https://github.com/brodeau/climporn
sys.path.append(CLIMPORN_DIR)
import climporn as cp


### Data

In [ ]:
fresults=sorted(glob.glob('results_'+name_sat+'-'+name_mod+'/result_??.nc'))
dsn=xr.open_mfdataset(fresults,concat_dim='time',combine='nested')

In [ ]:
print(name_mod)

In [ ]:
fresults=sorted(glob.glob('results/result_??.nc'))
dsn=xr.open_mfdataset(fresults,concat_dim='time',combine='nested')

In [ ]:
#clean up some remaining Nans

ds=dsn.where(nmp.isnan(dsn.sla_unfiltered)==0, drop=True)

### Maps of the tracks

In [ ]:
fig = plt.figure(num = 1, figsize=(9,9), facecolor='w', edgecolor='k')
ax = plt.subplot(111,projection=ccrs.PlateCarree())
ax.coastlines(resolution="10m")
ax.set_xticks(nmp.arange(-80,30,1.5), crs=ccrs.PlateCarree())
ax.set_xticklabels(nmp.arange(-80,30,1.5))
ax.set_yticks(nmp.arange(-50,50,1.5), crs=ccrs.PlateCarree())
ax.set_yticklabels(nmp.arange(-50,50,1.5))
ax.yaxis.tick_left()
ax.xaxis.tick_bottom()

lon_formatter = cticker.LongitudeFormatter()
lat_formatter = cticker.LatitudeFormatter()
ax.xaxis.set_major_formatter(lon_formatter)
ax.yaxis.set_major_formatter(lat_formatter)
gl = ax.gridlines(draw_labels=True, linestyle=':', color='black',
                          alpha=0.5)

idx = ds.longitude
idy = ds.latitude
t = ds.time_counter
cf = ax.scatter(idx, idy, c=t, cmap = 'viridis', marker='.', s=3 )
    
cbar = plt.colorbar(cf,shrink=0.8)
cbar.set_ticks([])

plt.savefig('plots/tracks_'+name_sat+'-'+name_mod+'.png')

### Time series of SSH 

In [ ]:
del fig, ax

clr_sat = '#AD0000'
clr_mod = '#008ab8'

VT = ds.time_counter ; # time from UNIX Epoch to Matlplotlib friendly...

fig = plt.figure(num = 1, figsize=(20,12), facecolor='w', edgecolor='k')
ax = plt.subplot(111)
plt.hlines(0,nmp.min(ds.time_counter.values),nmp.max(ds.time_counter.values), colors='k',label=None,  zorder=5)
plt.plot(VT, ds.sla_unfiltered-nmp.mean(ds.sla_unfiltered), '.', color=clr_sat, markersize=5, 
         alpha=0.5, label=name_sat, zorder=10)
plt.plot(VT, ds.sossheig_bl-nmp.mean(ds.sossheig_bl), '.', color=clr_mod, markersize=5, 
         alpha=0.5, label=name_mod, zorder=15)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
plt.xticks(rotation='60')
plt.ylabel('SSH [m]')
ax.grid(color='k', linestyle='-', linewidth=0.3)
lgnd = plt.legend(bbox_to_anchor=(0.55, 1.), ncol=1, shadow=True, fancybox=True)

plt.savefig('plots/SSH_'+name_sat+'-'+name_mod+'.png')

### Spectrum

In [ ]:
%%time

ISeg_beg, ISeg_end = gz.FindUnbrokenSegments( ds.time, ds.distance, ds.sossheig_bl, \
                                             rcut_time=1.2e+09, rcut_dist=7.8 )
NbSeg, Nsl, IDEDSeg = gz.SegmentSelection(ISeg_beg, ISeg_end, np_valid_seg=70)
XPs, XPm, rdist_sample = gz.Process4FFT( IDEDSeg, ds.distance, ds.sossheig_bl, ds.sla_unfiltered )
Kwn, PwSpc_s, PwSpc_m = gz.ApplyFFT( IDEDSeg, XPs, XPm, rdist_sample )

# Building our spectrum as the mean of the NbSeg spectra:
vps_mod = nmp.mean(PwSpc_m[:,:],axis=0)
vps_sat = nmp.mean(PwSpc_s[:,:],axis=0)

# Blabla for the plot:
cinfrm = str(NbSeg)+' segments\n'+str(Nsl)+' points/segment\n'+r'$\Delta$d sat.: '+str(round(rdist_sample,1))+' km'

ii = cp.plot("pow_spectrum_ssh")(Kwn, vps_mod, clab1=name_mod+' ("'+name_ssh_mod+'")', clr1=clr_mod, lw1=5, \
                                 cinfo=cinfrm, logo_on=False, \
                                 L_min=13., L_max=500., P_min_y=-6, P_max_y=1, \
                                 l_show_k4=False, l_show_k5=True, l_show_k11o3=False, l_show_k2=True, \
                                 vk2=Kwn, vps2=vps_sat, clab2=name_sat+' ("'+name_ssh_sat+'")', clr2=clr_sat, lw2=4)

plt.savefig('plots/spectrum_SSH_'+name_sat+'-'+name_mod+'.png')